In [22]:
import numpy as np
import h5py
import torch
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

# Helper Clases / Functions

In [71]:
def checkAccuracy(model, dataloader):
    y_pred_tot = []
    y_sample_tot = []
    for i_batch, sample_batched in enumerate(dataloader):
        X_sample, y_sample = sample_batched
        # Flip axis first
        X_sample = torch.Tensor(np.moveaxis(X_sample.numpy(), -1, 0))
        X_sample, y_sample = Variable(X_sample), Variable(y_sample)
        scores, _ = model(X_sample)
        #loss = loss_fn(scores, y_sample.long())
        #print('val loss: %f' % loss)
        _, y_pred = torch.max(scores, 0)
        _, y_pred = torch.max(y_pred, 1)
        y_pred_tot.append(y_pred.data.numpy())
        y_sample_tot.append(y_sample.data.numpy())
    acc = np.mean(np.concatenate(y_pred_tot) == np.concatenate(y_sample_tot))
    return acc
class Flatten(nn.Module):
    def forward(self, x):
        N, E, T = x.size() 
        return x.view(N, -1)

# Load Data

In [37]:
A01T = h5py.File('./../project_datasets/A01T_slice.mat', 'r')
print (A01T)
X = np.copy(A01T['image'])
y = np.copy(A01T['type'])
X = X[:, 0:23, :] # only goes to 23 since you dont use last 3 channels
# X = np.moveaxis(X, -1, 0)
y = y[0,0:X.shape[0]:1]
#subtract 769 so classes start at 0
y = y - 769
N, E, T = np.shape(X)
print (np.shape(X))

<HDF5 file "A01T_slice.mat" (mode r)>
(288, 23, 1000)


# Make DataLoaders

In [38]:
bs = 50
data = data_utils.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dset = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [N-100, 50, 50])
dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=bs, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}

# Define Model

In [30]:
# Working Model
dtype = torch.FloatTensor
model = nn.Sequential(
    Flatten(),
    nn.Linear(E*T, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, 4),
)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [39]:
dtype = torch.FloatTensor
channels = E;
classes = 4;
model = nn.Sequential(
    nn.LSTM(channels, classes, 2)
)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training

In [72]:
num_epochs = 10
for epoch in range(num_epochs):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        # Flip axis first
        X_train = torch.Tensor(np.moveaxis(X_train.numpy(), -1, 0))
        X_train, y_train = Variable(X_train), Variable(y_train)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs, _ = model(X_train)
        _, y_pred = torch.max(outputs, 0)
        y_pred = Variable(y_pred.data, requires_grad=True)
        loss = loss_fn(y_pred.type(dtype), y_train.long())        
        print('(%d batch) loss: %f' % (i, loss))
        loss.backward()
        optimizer.step()
    # print accuracy
    train_acc = checkAccuracy(model, dataloaders['train'])
    val_acc = checkAccuracy(model, dataloaders['val'])
    print('(Epoch %d / %d) train_acc: %f; val_acc: %f' % (epoch+1, num_epochs, train_acc, val_acc))

(0 batch) loss: 301.991699
(1 batch) loss: 260.270081
(2 batch) loss: 286.477417
(3 batch) loss: 298.451324
(Epoch 1 / 10) train_acc: 0.234043; val_acc: 0.200000
(0 batch) loss: 294.685944
(1 batch) loss: 335.728973
(2 batch) loss: 259.954468
(3 batch) loss: 243.674728
(Epoch 2 / 10) train_acc: 0.234043; val_acc: 0.200000
(0 batch) loss: 265.360107
(1 batch) loss: 278.817749
(2 batch) loss: 290.515015
(3 batch) loss: 316.933350
(Epoch 3 / 10) train_acc: 0.234043; val_acc: 0.200000
(0 batch) loss: 268.079285
(1 batch) loss: 280.342224
(2 batch) loss: 281.018982
(3 batch) loss: 323.844391
(Epoch 4 / 10) train_acc: 0.234043; val_acc: 0.200000
(0 batch) loss: 238.399963
(1 batch) loss: 354.941132
(2 batch) loss: 225.419556
(3 batch) loss: 337.896729
(Epoch 5 / 10) train_acc: 0.234043; val_acc: 0.200000
(0 batch) loss: 241.829117
(1 batch) loss: 374.296509
(2 batch) loss: 238.875412
(3 batch) loss: 290.212097
(Epoch 6 / 10) train_acc: 0.234043; val_acc: 0.200000
(0 batch) loss: 312.120789
(

In [9]:
y_pred = model(X_train)
loss = loss_fn(last_output, y_train)
print (loss)
model

NameError: name 'last_output' is not defined